In [1]:
! pip install openai
! pip install tiktoken
! pip install pandas
! pip install tqdm

In [2]:
import openai
import pandas as pd
import numpy as np
import time as t
import tiktoken
import csv

import pandas as pd
import numpy as np
import csv
from openai import OpenAI
from openai._client import OpenAI

In [3]:
client = OpenAI(
    api_key="YOUR KEYS",
)

In [4]:
import os
import openai
os.environ['OPENAI_API_KEY'] = "YOUR KEYS"
if 'OPENAI_API_KEY' in os.environ:
    print("API Key set successfully.")
else:
    print("Failed to set API Key.")

API Key set successfully.


In [5]:
# file_loc = "/home/local/ASURITE/anirmal1/Downloads/gab_dataset.csv"
# file_loc = 'youtube_dataset.csv'
# file_loc = 'twitter_dataset.csv'
# file_loc = 'gab_dataset.csv'
# file_loc = 'reddit_dataset.csv'
file_loc = 'Rationales_file_IMPLICIT_hatespeech_dataset.csv'
dataset = pd.read_csv(file_loc)
dataset

,post,class,ChatGPT Response,ChatGPT_Rationales,exp_split,label
0,- islam = animals,implicit_hate,"{\n ""text"": ""- islam = animals"",\n ""rational...",Rationales: NA; Derogatory Language: islam; Cu...,train,1
1,#altright is on fire . dream big meme big &,not_hate,"{\n ""text"": ""#altright is on fire . dream big...",Rationales: NA; Derogatory Language: NA; Cuss ...,train,0
2,jobs to americans first .,implicit_hate,"{\n ""text"": ""jobs to americans first ."",\n ""...",Rationales: NA; Derogatory Language: NA; Cuss ...,train,1
3,enough illegal means illegal ! after the rape...,implicit_hate,"{\n ""text"": ""enough illegal means illegal ! a...",Rationales: rape of the 14 yr . old; Derogator...,train,1
4,white killers live . black people don't . why ...,implicit_hate,"{\n ""text"": ""white killers live . black peopl...","Rationales: white killers, black people; Derog...",train,1
...,...,...,...,...,...,...
20386,"white nationalists are not a "" hate "" "" grou...",not_hate,"{\n ""text"": ""white nationalists are not a \""h...",Rationales: NA; Derogatory Language: NA; Cuss ...,test,0
20387,frc's on npr's dianne rehm show right now disc...,not_hate,"{\n ""text"": ""frc's on npr's dianne rehm show ...",Rationales: NA; Derogatory Language: NA; Cuss ...,test,0
20388,"thr shld be no sanctuary cities,counties or st...",implicit_hate,"{\n ""text"": ""thr shld be no sanctuary cities,...",Rationales: NA; Derogatory Language: NA; Cuss ...,test,1
20389,antifa hate whites . many are self hating whit...,implicit_hate,"{\n ""text"": ""antifa hate whites . many are se...","Rationales: NA; Derogatory Language: whites, s...",test,1


In [6]:
# test_df = dataset.head(20)

In [7]:
# test_df

# Prompting the API

In [8]:
def get_classification(prompt, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "Follow the instructions as closely as possible. Output exactly in the format that is specified by the user."},
            {"role": "user", "content": "You are a hate speech detection bot. Given a text, respond with the classification label towards given text with either hateful labeled as 1 or non-hateful labeled as 0. Note: It is essential to give classification for all the texts."},
            {"role": "assistant", "content": "Text: always thought it astounding no policing agency dares do this to the <number> armed muslim compounds all over the country"},
            {"role": "assistant", "content": "1"},
            {"role": "user", "content": prompt}
        ],
        model=model,
        temperature=0.1,
        max_tokens=500,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


# Testing for one example

In [9]:
# prompt = f'''{
# test_df['text'][3]
# }'''

In [10]:
# prompt

In [11]:
# response = get_classification(prompt)

In [12]:
# response

In [13]:
# prompt = f'''{
# test_df['text'][5]
# }'''

In [14]:
# prompt

In [15]:
# response = get_classification(prompt)

In [16]:
# response

# Testing for all samples

In [17]:
import time
from tqdm import tqdm

# Create a new CSV file to write the updated data
updated_file_path = 'implicit_dataset_llm_classification.csv'
with open(updated_file_path, mode='a', newline='') as updated_file:
    csv_writer = csv.writer(updated_file)

    # Write the header row
    csv_writer.writerow(dataset.columns.tolist() + ['Generated_Label'])

    # Iterate through the rows and generate stances
    for index, row in tqdm(dataset.iterrows(), total=len(dataset)):
        try:
            tweet = row['post']
            generated_label = get_classification(tweet)
        except Exception as e:
            print(index, e)
            generated_label = 'NA'
            # time.sleep(60)
        row_values = row.tolist()
        row_values.append(generated_label)
        csv_writer.writerow(row_values)

updated_data = pd.read_csv(updated_file_path)

100%|██████████| 20391/20391 [2:06:01<00:00,  2.70it/s]


In [18]:
updated_data

,post,class,ChatGPT Response,ChatGPT_Rationales,exp_split,label,Generated_Label
0,- islam = animals,implicit_hate,"{\n ""text"": ""- islam = animals"",\n ""rational...",Rationales: NA; Derogatory Language: islam; Cu...,train,1,1
1,#altright is on fire . dream big meme big &,not_hate,"{\n ""text"": ""#altright is on fire . dream big...",Rationales: NA; Derogatory Language: NA; Cuss ...,train,0,0
2,jobs to americans first .,implicit_hate,"{\n ""text"": ""jobs to americans first ."",\n ""...",Rationales: NA; Derogatory Language: NA; Cuss ...,train,1,0
3,enough illegal means illegal ! after the rape...,implicit_hate,"{\n ""text"": ""enough illegal means illegal ! a...",Rationales: rape of the 14 yr . old; Derogator...,train,1,1
4,white killers live . black people don't . why ...,implicit_hate,"{\n ""text"": ""white killers live . black peopl...","Rationales: white killers, black people; Derog...",train,1,1
...,...,...,...,...,...,...,...
20386,"white nationalists are not a "" hate "" "" grou...",not_hate,"{\n ""text"": ""white nationalists are not a \""h...",Rationales: NA; Derogatory Language: NA; Cuss ...,test,0,1
20387,frc's on npr's dianne rehm show right now disc...,not_hate,"{\n ""text"": ""frc's on npr's dianne rehm show ...",Rationales: NA; Derogatory Language: NA; Cuss ...,test,0,0
20388,"thr shld be no sanctuary cities,counties or st...",implicit_hate,"{\n ""text"": ""thr shld be no sanctuary cities,...",Rationales: NA; Derogatory Language: NA; Cuss ...,test,1,1
20389,antifa hate whites . many are self hating whit...,implicit_hate,"{\n ""text"": ""antifa hate whites . many are se...","Rationales: NA; Derogatory Language: whites, s...",test,1,1


In [19]:
from google.colab import files
files.download('implicit_dataset_llm_classification.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>